In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install polars

In [3]:
import os
import gc
import random
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [4]:
VER = "08"
LOCALES = ["DE", "UK", "JP"]

In [5]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/train_task1.parquet")
test1_1 = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/test_task1_phase1.parquet")
test1_2 = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/test_task1_phase2.parquet")
test3_1 = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task3/test_task3_phase1.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_2 = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task3/test_task3_phase2.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test1_1, test1_2, test3_1, test3_2])
product = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/common/product_04.parquet")

In [6]:
def preprocess(session_df: pl.DataFrame, product_df: pl.DataFrame) -> pl.DataFrame:
    session_df = session_df.explode("prev_items")
    session_df = session_df.join(product_df, left_on=["prev_items", "locale"], right_on=["id", "locale"], how="left")
    return session_df

In [8]:
def generate_product_features(session_df: pl.DataFrame, product_df: pl.DataFrame):
    product_features_df = product_df[["id", "locale", "price", "brand", "color", "size", "model", "material", "author"]]

    # Basic features of items
    product_features_df = product_features_df.rename({
        "price":"P_price",
        "brand":"P_brand",
        "color":"P_color",
        "size":"P_size",
        "model":"P_model",
        "material":"P_material",
        "author":"P_author",
    })

    # how many locales the item is deployed in
    product_features_df = product_features_df.join(
        product_df.groupby("id").count().rename({"count":"P_n_unique_locale"}),
        on="id",
        how="left"
    )

    # Number of items purchased
    product_features_df = product_features_df.join(
        session_df.groupby(["prev_items", "locale"]).count().rename({"count":"P_purchase_count"}),
        left_on=["id", "locale"],
        right_on=["prev_items", "locale"],
        how="left"
    )

    # Item purchase count (global)
    product_features_df = product_features_df.join(
        session_df.groupby(["prev_items"]).count().rename({"count":"P_purchase_count_global"}),
        left_on=["id"],
        right_on=["prev_items"],
        how="left"
    )

    # Number of times a product is purchased x product price
    product_features_df = product_features_df.with_columns(
        (pl.col("P_price") * pl.col("P_purchase_count")).alias("P_total_amount")
    )

    # non-null for each item
    is_null_df = product_df.with_columns(
        pl.col("color").is_null().cast(pl.UInt8).alias("P_is_color_null"),
        pl.col("size").is_null().cast(pl.UInt8).alias("P_is_size_null"),
        pl.col("model").is_null().cast(pl.UInt8).alias("P_is_model_null"),
        pl.col("material").is_null().cast(pl.UInt8).alias("P_is_material_null"),
        pl.col("author").is_null().cast(pl.UInt8).alias("P_is_author_null"),
    )[["id", "locale", "P_is_color_null", "P_is_size_null", "P_is_model_null", "P_is_material_null", "P_is_author_null"]]
    product_features_df = product_features_df.join(is_null_df, on=["id", "locale"], how="left")

    # Number of purchases of the brand to which the item belongs
    product_features_df = product_features_df.join(
        session_df.groupby(["brand", "locale"]).count().rename({"count":"P_brand_purchase_count"}),
        left_on=["P_brand", "locale"],
        right_on=["brand", "locale"],
        how="left",
    )

    # Number of purchases of the brand to which the item belongs (global)
    product_features_df = product_features_df.join(
        session_df.groupby(["brand"]).count().rename({"count":"P_brand_purchase_count_global"}),
        left_on=["P_brand"],
        right_on=["brand"],
        how="left",
    )

    # Number of purchases of the brand to which the item belongs (global)
    product_features_df = product_features_df.join(
        session_df.groupby(["brand"]).agg(pl.col("price").sum().alias("P_total_brand_amount")),
        left_on=["P_brand"],
        right_on=["brand"],
        how="left",
    )

    # Average, maximum, minimum, and standard deviation of the price of the brand of the item to which the product belongs
    product_features_df = product_features_df.join(
        product_df.groupby(["locale", "brand"]).agg(
            pl.col("price").mean().alias("P_brand_mean_price"),
            pl.col("price").max().alias("P_brand_max_price"),
            pl.col("price").min().alias("P_brand_min_price"),
            pl.col("price").std().alias("P_brand_std_price"),
        ),
        left_on=["locale", "P_brand"],
        right_on=["locale", "brand"],
        how="left"
    )

    # The difference between the product's price and the brand's average price
    product_features_df = product_features_df.with_columns(
        (pl.col("P_price") - pl.col("P_brand_mean_price")).alias("P_price_diff_to_avg_brand_price")
    )

    # Number of purchases for the locale to which the item belongs
    product_features_df = product_features_df.join(
        session_df.groupby(["locale"]).count().rename({"count":"P_locale_purchase_count"}),
        on=["locale"],
        how="left",
    )

    # Total purchase amount for the locale to which the item belongs
    product_features_df = product_features_df.join(
        session_df.groupby(["locale"]).agg(pl.col("price").sum().alias("P_total_locale_amount")),
        on=["locale"],
        how="left",
    )

    # Sales volume and sales amount of that product vs. sales volume and sales amount of the group
    product_features_df = product_features_df.with_columns([
        (pl.col("P_purchase_count") / (pl.col("P_locale_purchase_count") + 1)).alias("P_purchase_count_ratio_to_locale"),
        (pl.col("P_total_amount") / (pl.col("P_total_locale_amount") + 1)).alias("P_purchase_amount_ratio_to_locale"),
        (pl.col("P_purchase_count") / (pl.col("P_brand_purchase_count") + 1)).alias("P_purchase_count_ratio_to_brand"),
        (pl.col("P_total_amount") / (pl.col("P_total_brand_amount") + 1)).alias("P_purchase_amount_ratio_to_brand"),
    ])

    return product_features_df

# train/eval

In [9]:
session_df = pl.concat([
    train[["prev_items", "locale"]],
    test[["prev_items", "locale"]],
])

In [10]:
session_df = preprocess(session_df, product)

In [11]:
product_features = generate_product_features(session_df, product)

<ipython-input-8-2a25178b5aa9>:17: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  product_df.groupby("id").count().rename({"count":"P_n_unique_locale"}),
<ipython-input-8-2a25178b5aa9>:24: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["prev_items", "locale"]).count().rename({"count":"P_purchase_count"}),
<ipython-input-8-2a25178b5aa9>:32: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["prev_items"]).count().rename({"count":"P_purchase_count_global"}),
<ipython-input-8-2a25178b5aa9>:55: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["brand", "locale"]).count().rename({"count":"P_brand_purchase_count"}),
<ipython-input-8-2a25178b5aa9>:63: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["brand"]).count().rename({"count":"P_brand_purchase_

In [12]:
product_features.write_parquet(f"/content/drive/MyDrive/kddcup2023/data/interim/features/task1/product_feature_train_{VER}.parquet", use_pyarrow=True)

# test

In [13]:
# Append next_item of train to prev_items
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)
train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [14]:
session_df = pl.concat([
    train[["prev_items", "locale"]],
    test[["prev_items", "locale"]],
])

In [15]:
session_df = preprocess(session_df, product)

In [16]:
product_features = generate_product_features(session_df, product)

<ipython-input-8-2a25178b5aa9>:17: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  product_df.groupby("id").count().rename({"count":"P_n_unique_locale"}),
<ipython-input-8-2a25178b5aa9>:24: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["prev_items", "locale"]).count().rename({"count":"P_purchase_count"}),
<ipython-input-8-2a25178b5aa9>:32: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["prev_items"]).count().rename({"count":"P_purchase_count_global"}),
<ipython-input-8-2a25178b5aa9>:55: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["brand", "locale"]).count().rename({"count":"P_brand_purchase_count"}),
<ipython-input-8-2a25178b5aa9>:63: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  session_df.groupby(["brand"]).count().rename({"count":"P_brand_purchase_

In [19]:
product_features.write_parquet(f"/content/drive/MyDrive/kddcup2023-master/data/interim/features/task1/product_feature_test_{VER}.parquet", use_pyarrow=True)

FileNotFoundError: [Errno 2] Failed to open local file '/content/drive/MyDrive/kddcup2023-master/data/interim/features/task1/product_feature_test_08.parquet'. Detail: [errno 2] No such file or directory